In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

%matplotlib inline

In [2]:
combined_pre = pd.read_csv('../assets/combined_pre_classed.csv', index_col= 0)
combined_post = pd.read_csv('../assets/combined_post_classed.csv', index_col=0)

In [3]:
combined_post.head()

,text,disaster
0,live in rd ward at peggy park giving bacc to...,0
1,accident cleared in bellaire on w lp nb ...,0
2,a nice cold saintarnoldbrewing art car ipa af...,0
3,where to shop sale of unique items helps aid...,1
4,aerial tour shows devastation of epic floodin...,1


In [4]:
combined_pre.head()

,text,disaster
0,baytown pd man sought after teen suspect kil...,0
1,amy st houston tx,0
2,crazy about this sisterhood stephen f austi...,0
3,drinking a steam punk rousing steam lager by ...,0
4,reposted with the right cosplayer this time ...,0


In [5]:
combined_post.disaster.value_counts()

0    44896
1    22231
Name: disaster, dtype: int64

In [6]:
disaster_keywords = []
with open('../assets/disaster_keywords.txt') as f:
    for phrase in f:
        disaster_keywords.append(phrase.strip())
disaster_keywords[:5]

['earthquake', 'hurricane', 'fire', 'emergency', 'help']

In [7]:
add_stop = ['rt', 'twitter', 'com', 'net', 'carr', 'irma', 'harvey',
            'maria', 'carrfire', 'camp', 'campfire', 'woolsey', 
            'woolseyfire', 'prague', 'north', 'northbay', 'news',
            'hurricaneirma', 'hurricaneharvey', 'hurricanemaria',
            'headline', 'breaking']

In [8]:
disaster_set = set(disaster_keywords)
disaster_list = list(disaster_set)

In [9]:
disaster_list.extend(add_stop)

In [10]:
stopwords = nltk.corpus.stopwords.words('english')

In [11]:
stopwords.extend(disaster_list)

In [12]:
combined_df = pd.concat([combined_pre, 
                        combined_post], 
                       ignore_index=True)

In [13]:
combined_df.shape

(98550, 2)

In [14]:
combined_df.head(2)

,text,disaster
0,baytown pd man sought after teen suspect kil...,0
1,amy st houston tx,0


In [15]:
combined_df.disaster.value_counts(normalize=True)

0    0.774419
1    0.225581
Name: disaster, dtype: float64

In [16]:
y = combined_df['disaster']

# Set X as text column.
X = combined_df['text']

In [40]:
tfidf = TfidfVectorizer(stop_words = stopwords, 
                        max_df=0.95, 
                        min_df=5, max_features=10000)

In [41]:
combined_tfidf = tfidf.fit_transform(X)

/Users/jameslee/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['beaufort', 'bermuda', 'dust', 'high', 'low', 'pressure', 'richter', 'scale', 'triangle', 'vane', 'violent'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [42]:
combined_df_tfidf = pd.SparseDataFrame(combined_tfidf,
                             columns=tfidf.get_feature_names())

In [43]:
combined_df_tfidf.shape

(98550, 10000)

In [44]:
import sys

In [45]:
sys.getsizeof(combined_df_tfidf) * 1e-9

0.011253576000000001

In [46]:
combined_df_tfidf.head()

,__,aa,aaa,aan,aaron,aaroncarter,ab,abandoned,abatido,abby,...,zswaggers,zswagtour,zt,zu,zumba,zv,zw,zx,zy,zz
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
combined_df_tfidf.fillna(0, inplace=True)

In [48]:
combined_df_tfidf.sum().sort_values(ascending = False).head(20)

florida       4685.343695
keys          4240.856064
pic           2697.864702
california    1886.096603
houston       1882.239273
oaks          1818.031640
thousand      1744.670375
key           1317.453544
shooting      1314.783959
bar           1117.782834
texas         1100.405905
people         987.690306
west           980.511379
landfall       829.844184
via            760.199409
like           750.710482
category       701.086951
love           682.817763
get            661.755415
mass           661.354474
dtype: float64

In [49]:
y.shape

(98550,)

In [50]:
combined_df_tfidf.shape

(98550, 10000)

In [51]:
combined_array = np.asarray(combined_df_tfidf)

In [54]:
sys.getsizeof(combined_array)

112

In [52]:
sys.getsizeof(combined_array) * 1e-9

1.1200000000000001e-07

In [ ]:
# Train/test split our data.
# X_train, X_test, y_train, y_test = train_test_split(X,
#                                                     y,
#                                                     test_size = 0.33,
#                                                     random_state = 42)

In [55]:
# Import PCA.
from sklearn.decomposition import PCA

In [56]:
# Instantiate PCA.
pca = PCA(n_components=100)

In [57]:
pd.DataFrame(combined_array).shape

(98550, 10000)

In [ ]:
# Fit PCA on the training data.
Z_combined = pca.fit_transform(combined_array)

# n_component will default to number of features in this case 9999 features.

In [38]:
pd.DataFrame(Z_combined).describe()

# column 0 is the first principal component

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
count,9.855000e+04,9.855000e+04,9.855000e+04,9.855000e+04,9.855000e+04,9.855000e+04,9.855000e+04,9.855000e+04,9.855000e+04,9.855000e+04,...,9.855000e+04,9.855000e+04,9.855000e+04,9.855000e+04,9.855000e+04,9.855000e+04,9.855000e+04,9.855000e+04,9.855000e+04,9.855000e+04
mean,-1.778606e-15,-1.010376e-15,-4.189620e-16,-9.012930e-17,-7.028897e-17,1.597978e-16,8.339843e-17,-2.131758e-16,-9.182959e-17,2.702423e-17,...,5.062048e-18,2.530024e-17,1.436285e-17,1.093545e-17,-5.541969e-17,-1.895483e-17,9.368444e-18,-2.096613e-17,1.652707e-17,4.025597e-19
std,1.650207e-01,1.402026e-01,1.190333e-01,1.152824e-01,9.929207e-02,8.122586e-02,7.848397e-02,7.489747e-02,7.280805e-02,7.225260e-02,...,4.182133e-02,4.164577e-02,4.147493e-02,4.145434e-02,4.122584e-02,4.115070e-02,4.090029e-02,4.082382e-02,4.058531e-02,4.023560e-02
min,-4.537709e-01,-6.122365e-01,-1.691763e-01,-2.467681e-01,-2.595318e-01,-2.737119e-01,-1.931123e-01,-6.342293e-01,-3.328845e-01,-5.230166e-01,...,-2.576357e-01,-3.348824e-01,-2.701302e-01,-2.466682e-01,-2.320357e-01,-2.838890e-01,-2.941234e-01,-2.629867e-01,-2.785617e-01,-2.292427e-01
25%,-7.326945e-02,-2.810670e-02,-9.020374e-02,-6.105505e-02,-3.671743e-02,-2.145890e-02,-2.338607e-02,-5.491663e-03,-2.188994e-02,-7.039934e-03,...,-1.790846e-02,-1.658479e-02,-1.597144e-02,-1.845799e-02,-1.779103e-02,-1.783090e-02,-1.793489e-02,-1.690202e-02,-1.679546e-02,-1.819264e-02
50%,-5.886256e-02,1.101424e-02,-2.435538e-02,-1.047768e-02,-2.221602e-02,4.776592e-04,-5.437872e-03,6.003797e-04,2.718021e-04,-1.817432e-03,...,-3.576961e-03,1.391813e-03,2.642997e-03,1.298277e-03,-1.868363e-03,-3.417279e-03,-1.719528e-03,1.793307e-03,3.450941e-03,7.620825e-04
75%,1.265291e-01,3.890583e-02,5.357590e-02,-3.932853e-03,-3.982950e-03,4.613698e-03,4.999671e-03,5.792153e-03,9.349469e-03,7.171680e-03,...,1.608945e-02,1.638583e-02,1.593548e-02,1.775526e-02,1.503107e-02,1.688448e-02,1.618591e-02,1.698397e-02,1.813620e-02,1.590833e-02
max,7.564889e-01,6.902804e-01,6.468355e-01,9.442297e-01,8.762766e-01,7.320055e-01,8.423308e-01,7.004911e-01,7.105983e-01,8.021622e-01,...,3.386977e-01,3.713849e-01,4.458523e-01,3.145512e-01,3.606883e-01,3.316814e-01,3.226459e-01,3.071840e-01,2.891618e-01,2.649695e-01


In [39]:
var_exp = pca.explained_variance_ratio_
print(f'Explained variance: {np.round(var_exp,3)}')

cum_var_exp = np.cumsum(var_exp)
print(f'Cumulative explained variance: {np.round(cum_var_exp,3)}')

Explained variance: [0.029 0.021 0.015 0.014 0.01  0.007 0.006 0.006 0.006 0.005 0.005 0.005
 0.005 0.005 0.005 0.004 0.004 0.004 0.004 0.004 0.004 0.004 0.004 0.004
 0.004 0.004 0.004 0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.003
 0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.003
 0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002
 0.002 0.002 0.002 0.002]
Cumulative explained variance: [0.029 0.049 0.064 0.078 0.089 0.096 0.102 0.108 0.114 0.119 0.125 0.129
 0.134 0.139 0.143 0.148 0.152 0.156 0.16  0.164 0.168 0.172 0.176 0.18
 0.183 0.187 0.19  0.194 0.197 0.201 0.204 0.207 0.21  0.214 0.217 0.22
 0.223 0.226 0.229 0.232 0.235 0.238 0.24  0.243 0.246 0.249 0.252 0.254
 0.257 0.26  0.262 0.265 0.268 0.27  0.273 0.275 